# Libs & Settings

In [1]:
import sys
import warnings
import numpy as np
import polars as pl
import pandas as pd
import xgboost as xgb
from numba import njit
import lightgbm as lgb
from pathlib import Path
from mlforecast import MLForecast
from window_ops.rolling import rolling_mean
from sklearn.metrics import mean_squared_error
from window_ops.expanding import expanding_mean
from dateutil.relativedelta import relativedelta
# from hierarchicalforecast.units import aggregate
from sklearn.ensemble import RandomForestRegressor
from mlforecast.target_transforms import Differences

# To apply reconciliation
# from hierarchicalforecast.core import HierarchicalReconciliation
# from hierarchicalforecast.methods import BottomUp, TopDown, ERM, OptimalCombination

In [15]:
# In case of problems with loading from "hierarchicalforecast.units" run this cell after substituting the corresponding path.
%run -i "/Users/ivanandrusin/Desktop/PyCharmProjects/data-wagon-pioneers/.venv/lib/python3.9/site-packages/hierarchicalforecast/utils.py"

In [2]:
# Import user modules.
module_path = str(Path.cwd().parents[0])

if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_process import (
    add_master_data_mappings,
    process_ts,
    get_unique_id,
    ts_length,
    evaluate,
    calc_score_public,
    calc_score_private   
)
from utils.forecast_prep import forecast_prep
from model.train_pred import tarin_predict, get_detalization

In [3]:
# Folder with for keeping data files.
data_folder = Path().cwd().parent / 'data'

# Data Preparation

In [4]:
# Read data.
df = pd.read_csv(
    data_folder / "fact_train_test.csv", 
    sep=";", 
    decimal=",", 
    encoding="windows-1251"
)
# Convert datetime.
df["period"] = df["period"].astype("datetime64[ns]")

Прогнозирование осуществляется на грануляции данных, соответсвующей KPI:
- period
- rps
- holding_name
- sender_department_name
- recipient_department_name

In [5]:
# Add corresponding columns from mapper.
df_mp = add_master_data_mappings(df=df, data_folder=data_folder)
# Time series aggregation levels.
accuracy_granularity = [
    "period",
    "rps",
    "holding_name",
    "sender_department_name",
    "recipient_department_name",
]
# Aggregate data in accordance with given granularity.
df_mp_agg = (
    process_ts(
        df=df_mp, 
        grp=accuracy_granularity, 
        cutoff="20170101" # cut-off old data in order to increase efficiency
    )
    .set_index("period")
    .astype(np.int64)
    .reset_index()
)

Приведение всех временных рядов к одной длине посредством декартового произведения дат с уникальными id временых рядов:

In [6]:
# Time series alignment.
df_mp_agg_full = ts_length(
    df=df_mp_agg, 
    grp=accuracy_granularity, 
    exclude=['period']
)

# Forecast

In [7]:
# Train/test split date.
SPLIT_DATE = df_mp_agg_full['period'].max() - relativedelta(months=5)
# Create a list of the columns that represent the different levels of the hierarchy.
spec = [
    ['rps'],
    ['rps', 'holding_name'],
    ['rps', 'holding_name', 'sender_department_name'],
    ['rps', 'holding_name', 'sender_department_name', 'recipient_department_name']
]

## Model selection and tuning

Для прогноза используем библиотеки [mlforecast](https://github.com/Nixtla/mlforecast) и [hierarchicalforecast](https://github.com/Nixtla/hierarchicalforecast), которые имеют удобные пайплайны для тестирования различных моделей и реконсолидации иерархических временных рядов.

In [16]:
# Split the data into different levels
train, valid = forecast_prep(
		df=df_mp_agg_full,
		split_date=SPLIT_DATE,
		grp=accuracy_granularity,
)
train_agg, S_train, tags = aggregate(train, spec)
valid_agg, _, _ = aggregate(valid, spec)

In [8]:
@njit
def rolling_mean_3(x):
    return rolling_mean(x, window_size=3)

Выбор моделей и добавление признаков:

In [12]:
# Specify models.
mlf_models = [
    lgb.LGBMRegressor(verbosity=-1),
    xgb.XGBRegressor(),
    RandomForestRegressor(random_state=0),
]
# Define settings.
mlf = MLForecast(
    models=mlf_models,
    freq='MS',
    lag_transforms={
        2: [expanding_mean],
        2: [rolling_mean_3]
    },
    lags=[1, 2],
    # date_features=['month'],
    target_transforms=[Differences([1])],
)
# Fit model.
mlf.fit(df=train_agg.reset_index())

MLForecast(models=[LGBMRegressor, XGBRegressor, RandomForestRegressor], freq=<MonthBegin>, lag_features=['lag1', 'lag2', 'rolling_mean_3_lag2'], date_features=[], num_threads=1)

In [13]:
# Generate predictions.
base_forecast_df = mlf.predict(5)

In [14]:
# Forecast validation.
res = valid_agg.reset_index().merge(
    base_forecast_df, 
    how='left', 
    on=['unique_id', 'ds']
).fillna(0)
model_cols = [col for col in base_forecast_df.columns if col not in ['unique_id', 'ds']]
for col in model_cols:
    print(
        f"{col}: {mean_squared_error(res['y'], res[col], squared=False)/1e3}")

LGBMRegressor: 2.2451169882258593
XGBRegressor: 2.598655589396085
RandomForestRegressor: 2.5986660735015645


Реконсолидация временных рядов на разных уровнях (при текущем объеме данных требует достаточно много вычислительных ресурсов - минимум около 32 Гб RAM). При наличии достаточных ресурсов перенести код ниже в исполняемую ячейку.
```python
reconcilers = [
    BottomUp(),
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
base_forecast_df = hrec.reconcile(
    Y_hat_df=base_forecast_df.set_index('unique_id'), 
    Y_df = train_agg, 
    S=S_train, 
    tags=tags
)
```

## Get predictions

Выбираем модель с наименьшей ошибкой на тестовой выборке и обучаем на всем датасете:

In [9]:
# Select best model based on above research.
best_model = mlf = MLForecast(
    models=[lgb.LGBMRegressor(verbosity=-1)],
    freq='MS',
    lag_transforms={
        2: [expanding_mean],
        2: [rolling_mean_3]
    },
    lags=[1, 2],
    # date_features=['month'],
    target_transforms=[Differences([1])],
)
answer = tarin_predict(
    df=df_mp_agg_full, 
    grp=accuracy_granularity, 
    model=best_model,
    spec=spec,
    horizon=5
)

## Increase detalization

Увеличиваем детализацию временных рядов. Для этого определим два уровня агрегации врменных рядов, присвоим им уникальные id и найдем пропорцию вхождения каждого из низкоуровневых временных рядов (НВР) в высокоуровневый (ВВР). Далее распределим значения таргета, полученные после обучения модели для ВВР, между НВР в соответсвующих пропорциях.

In [10]:
# Set low-level unique id columns.
uid = [
    'rps', 
    'podrod',
    'filial',
    'client_sap_id',
    'freight_id',
    'sender_station_id',
    'recipient_station_id',
    'sender_organisation_id'
]

In [11]:
# Get final submission dataframe.
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    submit_df = get_detalization(
        df=df_mp, 
        answer=answer, 
        model_name='LGBMRegressor', 
        uid=uid,
        grp=accuracy_granularity,
        start_period='2023-01-01', 
        weight_factor=60
    )

In [12]:
# Save predictions file for validation/submission.
submit_df.to_csv(
    data_folder / "forecast_lgb_v1.csv", 
    index=False, sep=";", 
    decimal=",", 
    encoding="windows-1251"
)

# Validation

In [28]:
# Save test file for validation.
df[df['period'] > SPLIT_DATE].to_csv(
    data_folder / "fact_validation.csv",
    index=False, sep=";", 
    decimal=",", 
    encoding="windows-1251"
)

In [ ]:
# = Примеры файлов для проверки =
validation_file = data_folder / "fact_validation.csv"
forecast_file = data_folder / "forecast_lgb_v1.csv"

# Валидационный датасет
fact = pd.read_csv(validation_file, sep=";", decimal=",", encoding="windows-1251")
# print("Валидационный датасет:", fact.shape)
# Прогноз
forecast = pd.read_csv(forecast_file, sep=";", decimal=",", encoding="windows-1251")
# print("Прогноз:", forecast.shape)

# Скорим
score_public = calc_score_public(fact, forecast)
score_private = calc_score_private(fact, forecast)
print(f"Public score: {score_public}")
print(f"Private score: {score_private}")